**Task 1: Understand the problem and setup environment**

We're trying to track the trajectory of the ball object across the video. Here we need to detect the ball object first and keep track of that object across the video. 


**kalman Filter -** 

Kalman Filter is useful to predict the state of an object and track the object with the given measurements when there is noise included in the measurements.
It uses the prior knowledge of the state of the object 
Kalman Filter uses following equations - 
The process noise is used to model the uncertainty
**Covariance Equation**- uncertainity in the prediction is modeled using this equation 
**Prediction equation** - predicts the next system state based on the knowledge of the current state
**State Update Equation** -  is responsible for the system's current state estimation using the State Update equation
**Covariance update equation** - The state update process calculates the Kalman Gain and outputs Current System State Estimate and Current State Estimate Uncertainty, which will input the predicted process
**Kalman Gain** - computes a Kalman Gain for each new measurement to minimize the estimate variance 

 
 Kalman filter process has 2 steps - 
 Start with initialization, and this step is performed only once for the first measurement. Once initialized, the Kalman Filter will predict the system state at the next time step, and it also provides the uncertainty of the prediction
1.**Prediction step** -  the Kalman Filter predicts the next state of the system given the previous measurements.
Prediction step uses - State Transision Matrix and Control input matrix 
2.**Updation step**  - the Kalman Filter estimates the current state of the system given the measurement at that time step - uses Measurement Mapping Matrix and the predicted matrix is used to calculate the kalman Gain and update the state of the object. 

**Single Object tracking** - In this challenge, we're trying to track the single instance of a ball with occlusions. We can track the ball across different frames of the video by using a simple object detector but that fails when the ball goes behind the paper. 
We need to keep track of the location of the ball even when there is an occlusion. This can be done by using kalman filter. 
KF object is initialized with random values and gets updated after the first cycle. Then it keeps tracking the centers of the detected object.


**Multiple Object tracking** - In this challenge, we're trying to keep track of 2 ball objects in the same video. The challenge in this is to keep track of the trajectory of the each separately even after they cross paths with each other or even collide with each other and change directions. 
In this case Kalman filter can be used to keep track of the trajectories of each ball instances separately. 
Need to assign unique identity for 2 trackers to track each detected object separately. and use the kalman function's predict and update to keep track of the detected objects.

In [ ]:
!pip3 install tensorflow==2.2.0
!pip3 install keras==2.2.4
!pip3 install imageai --upgrade


     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 2.9 MB 33.2 MB/s 
     |████████████████████████████████| 454 kB 45.2 MB/s 
     |████████████████████████████████| 3.0 MB 40.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0+zzzcolab202205

In [ ]:
import numpy as np
import cv2
import tensorflow.keras
from imageai.Detection import ObjectDetection
import os
from google.colab.patches import cv2_imshow
import tensorflow as tf


In [ ]:
class KalmanFilter(object):
    def __init__(self, dt, a_x,a_y, std_acc, x_std_meas, y_std_meas):
        # Define sampling time
        self.dt = dt
        # Define the  control input variables
        self.u = np.matrix([[a_x],[a_y]])
        # Intial State
        self.x = np.matrix([[0], [0], [0], [0]])
        # Define the State Transition Matrix A
        self.A = np.matrix([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
        # Define the Control Input Matrix B
        self.B = np.matrix([[(self.dt**2)/2, 0],
                            [0, (self.dt**2)/2],
                            [self.dt,0],
                            [0,self.dt]])
        # Define Measurement Mapping Matrix
        self.H = np.matrix([[1, 0, 0, 0],
                            [0, 1, 0, 0]])
        #Initial Process Noise Covariance
        self.Q = np.matrix([[(self.dt**4)/4, 0, (self.dt**3)/2, 0],
                            [0, (self.dt**4)/4, 0, (self.dt**3)/2],
                            [(self.dt**3)/2, 0, self.dt**2, 0],
                            [0, (self.dt**3)/2, 0, self.dt**2]]) * std_acc**2
        #Initial Measurement Noise Covariance
        self.R = np.matrix([[x_std_meas**2,0],
                           [0, y_std_meas**2]])
        #Initial Covariance Matrix
        self.P = np.eye(self.A.shape[1])
    def predict_next_position(self):
        #State Transition Matrix and Control input matrix is used to predict the next state of the object
        self.x = np.dot(self.A, self.x) + np.dot(self.B, self.u)
        self.P = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        return self.x[0:2]
    def update_object_position(self, z):
        #Update the state of the object using Measurement Mapping Matrix and Predicted position from the predict function
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  
        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))   
        I = np.eye(self.H.shape[1])
        # Update error covariance matrix
        self.P = (I - (K * self.H)) * self.P   
        return self.x[0:2]

Part 2 - In this part we're detecting the ball object from each frame of the video. Here we are using YOLOV3 as object detection model. This model has been trained to detect objects using COCO dataset. This detects the ball object with class = "sports ball".

In [ ]:
#This method detects the ball object in the image provided, calculates the centroid of the object and returns the centroid of the detected object
def object_detection(image):
  current_directory = os.getcwd()
  detector = ObjectDetection() #Initializing the detector object  
  detector.setModelTypeAsYOLOv3() #Set the model type for the detector as YOLOV3
  detector.setModelPath('drive/MyDrive/attachments/yolo.h5') #set the model path
  detector.loadModel() #loading the model
  
  #detect the objects from the image
  detections = detector.detectObjectsFromImage(input_image = image, input_type = "array", output_image_path=os.path.join(current_directory , "output.jpg"))
  centers = [] #Array to store the centroid of the detected
  for eachObject in detections:
      x1,y1,x2,y2 = eachObject["box_points"] #get the bounding box coordinates of the detected object
      cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2) #draw rectangle around the detected object
      cv2.putText(image,"Bounding Box",(x1+15,y1+15), 0,0.5,(0,255,0), 2)
      cx = int((x1+x2)/2)
      cy = int((y1+y2)/2)
      inputCentroids = (cx,cy) #Calculating centroid of the bounding box
      centers.append(np.array([[cx],[cy]]))
  return centers #return the centroid of the detected object


**SINGLE OBJECT TRACKING-**
In this part, we're initializing the KF Object with initial values for time sampling time - time taken for 1 cycle. 
we're reading the video frame by frame using CV2 video capture. while the cv2.videoCapture method returns true, pass the frame through detect_object() method and get the centroid of the detected object.
if the centroid is detected then pass those to update function of the kalman filter. the use the predice function to predict the next state of the detected centroid.Predicts the next position of the centroid, initalised when KF object is created, converges to the real object position after first cycle This prediction works even when the object goes behind the paper. 
Finally we are using Cv2 video writer to add all the frames to a video. 

In [ ]:
import numpy as np
HiSpeed= 100
ControlSpeedVar = 30  
    #Create KalmanFilter object KF
    #KalmanFilter(dt, a_x, a_y, std_acc, x_std_meas, y_std_meas)
KF = KalmanFilter(0.1, 1, 1, 1, 0.1,0.1)
    # array to store the images to output to a video
img_array = []

VideoCap = cv2.VideoCapture('drive/MyDrive/attachments/ball.mp4')
while(True):
 
  ret, image = VideoCap.read()
        # Detect object
  if ret is False:
      break
  centers = object_detection(image)
  if (len(centers) > 0):
        # If centroids are detected then track the ball object
            # Draw the detected circle
    cv2.circle(image, (int(centers[0][0]), int(centers[0][1])), 10, (0, 191, 255), 2)
    cv2.putText(image, "Centroid", (centers[0][0] + 15, centers[0][1] - 15), 0, 0.5, (0,191,255), 2)
    # Update the centroid of the object
    (x1, y1) = KF.update_object_position(centers[0])
    #Draw a rectangle as the next estimated position of the object
    cv2.rectangle(image, (x1 - 15, y1 - 15), (x1 + 15, y1 + 15), (255, 0, 0), 2)
    cv2.putText(image, "Estimated position Position", (x1 + 15, y1), 0, 0.5, (255, 0, 0), 2)
            # Predict the next position of the centroid, initalised when KF object is created, converges to the real object position after first cycle
  (x, y) = KF.predict_next_position()
            # Draw a rectangle as the predicted object position
  cv2.rectangle(image, (x - 15, y - 15), (x + 15, y + 15), (255, 0, 0), 2)
  cv2.putText(image, "Predicted Position", (x + 15, y), 0, 0.5, (255, 0, 0), 2)  
  
  height, width, layers = image.shape
  size = (width,height)
  img_array.append(image)

  if cv2.waitKey(2) & 0xFF == ord('q'):
            VideoCap.release()
            cv2.destroyAllWindows()
            break
  cv2.waitKey(HiSpeed-ControlSpeedVar+1)

out = cv2.VideoWriter('single_object_tracking_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

MULTIPLE OBJECT TRACkING - Here we have 2 instances of the same object in the video. We are reading the video first and then detected the object instances using YOLOV3 pretrained model. 

In [ ]:
import numpy as np
import os
KF1 = KalmanFilter(0.1, 1, 1, 1, 0.1,0.1)
color = (255, 0, 0)
thickness = 2
current_directory = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('drive/MyDrive/attachments/yolo.h5')
detector.loadModel()
img_array = []
vidcap = cv2.VideoCapture('drive/MyDrive/attachments/multiple objects.avi')
success,frame = vidcap.read()
while success:
    ret,image = vidcap.read()
    
    if ret is False:
      break
    height, width, layers = image.shape
    size = (width,height)
    detections = detector.detectObjectsFromImage(input_image = image, input_type = "array", output_image_path=os.path.join(current_directory , "output.jpg"))
   
    for eachObject in detections:
      inputCentroids = np.zeros((len(detections), 2), dtype="int")
      x1,y1,x2,y2 = eachObject["box_points"]
      
     #Calculate the centroid of the object to 
      cx = int((x1+x2)/2)
      cy = int((y1+y2)/2)
      inputCentroids = (cx,cy)   

      cv2.rectangle(image,(x1,y1),(x2,y2), color, thickness) #bounding box of the detected object
      cv2.putText(image, "Bounding box", (x1+15,y1+15), 0, 0.5, (0,255,0), 2)
      cv2.circle(image, (cx,cy), 10, (0,0,255), thickness)
      cv2.putText(image, "Centroid", (cx + 15, cy - 15), 0, 0.5, (0,255,0), 2)
      
       
      img_array.append(image)
    if cv2.waitKey(2) & 0xFF == ord('q'):
     vidcap.release()
     cv2.destroyAllWindows()
     break
out = cv2.VideoWriter('Multiple_Obj_detection_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()